> **Note (Converted to IBM Runtime):**
> This notebook now targets the **IBM Runtime**. It:
> - Imports `QiskitRuntimeService, SamplerV2, EstimatorV2` from `qiskit_ibm_runtime`.
> - Adds a setup cell to create `service` and pick `backend = service.backend("simulator_stabilizer")`.
> - Builds primitives as `SamplerV2(mode=backend)` / `EstimatorV2(mode=backend)`.
> - Transpiles with `transpile(..., backend=backend)` for target-strict submission.
> - Sets **shots per `run()`** (Runtime no longer uses `default_shots`).
> Make sure your env has `QISKIT_IBM_CHANNEL/TOKEN/INSTANCE` set.


In [1]:
# IBM Runtime setup (requires env vars: QISKIT_IBM_CHANNEL/TOKEN/INSTANCE)
from qiskit_ibm_runtime import QiskitRuntimeService
service = QiskitRuntimeService() 
# Pick any available backend you have access to (real device or simulator):
backend = service.backends()[0]  # e.g., "ibm_sherbrooke" for a device, if you have access
print('the backends available are', service.backends())
print('using backend ', backend)


the backends available are [<IBMBackend('ibm_brisbane')>, <IBMBackend('ibm_torino')>]
using backend  <IBMBackend('ibm_brisbane')>


<div style="background:#0f62fe;color:#fff;padding:14px 16px;border-radius:8px;margin:8px 0;font-weight:700;">
Module 5.1 — Sampler Primitive Options
</div>

**Learning Objectives**

- Identify what `SamplerV2` returns and where options are set.
- Set default options on `sampler.options` and override per `run()`.
- Control **shots** and read the result object.
- Configure **Dynamical Decoupling (DD)** and **Pauli Twirling** (backend-dependent).
- Validate settings and understand simulator vs. hardware behavior.



---



<div style="background:#001d6c;color:#fff;padding:10px 12px;border-radius:6px;margin:8px 0;font-weight:600;">
5.1.1 — Sampler overview & where options live
</div>

**Overview**  
- `SamplerV2` returns **shot-based bitstring distributions** (probabilities/counts + metadata).
- **Defaults** live on `sampler.options`; per-execution overrides go in `run(...)`.
**Tip:** Construct → set defaults → `run()` with PUBs → inspect `.result()`.

**Example Code: Basics**

In [2]:
# Explanation: minimal pattern (requires qiskit & qiskit-ibm-runtime)
from qiskit import QuantumCircuit, transpile
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2, Options

service = QiskitRuntimeService() 
# Pick any available backend you have access to (real device or simulator):
# backend = service.backends()[1]  # e.g., "ibm_sherbrooke" for a device, if you have access
# print('the backends available are', service.backends())
# print('using backend ', backend)
qc = QuantumCircuit(2)
qc.h(0); qc.cx(0,1); qc.measure_all()

# 🔧 Make the circuit match the backend’s target
qc_native = transpile(qc, backend=backend, optimization_level=1)

sampler = SamplerV2(mode=backend)           # no Options(default_shots=...) here
job = sampler.run([qc_native], shots=4096)  # <-- set shots per call
res = job.result()
print(res[0])

SamplerPubResult(data=DataBin(meas=BitArray(<shape=(), num_shots=4096, num_bits=2>)), metadata={'circuit_metadata': {}})




---



<div style="background:#001d6c;color:#fff;padding:10px 12px;border-radius:6px;margin:8px 0;font-weight:600;">
5.1.2 — Setting & overriding options (workflow)
</div>

**Workflow**  
1) Choose backend/session  →  2) `SamplerV2()`  →  3) set `sampler.options`  →  4) override selectively in `run()`.

**Example Code: Defaults vs. Per‑run Override**

In [10]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import random_hermitian
from qiskit.circuit.library import IQP
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2

# --- config ---
n_qubits = 50
shots = 20_000

# --- pick a backend you actually have access to ---
service = QiskitRuntimeService()
sims = service.backends(simulator=True)
devs = service.backends(simulator=False)
print("Simulators:", [b.name for b in sims])
print("Devices:",    [b.name for b in devs])
PREFERRED = ["simulator_mps", "ibmq_qasm_simulator", "simulator_statevector"]
by_name = {b.name: b for b in sims}
backend = next((by_name[n] for n in PREFERRED if n in by_name), sims[0] if sims else devs[0])
print("Using backend:", backend.name)

# --- build a real symmetric matrix for IQP (qiskit 2.x: random_hermitian -> Operator) ---
H = random_hermitian(n_qubits, seed=1234)
A = np.asarray(getattr(H, "data", H.to_matrix()))          # complex ndarray
mat = np.real((A + A.T.conj()) / 2.0)                      # real symmetric (n x n)

# --- IQP circuit and measurements ---
circuit: QuantumCircuit = IQP(mat)
circuit = circuit.measure_all(inplace=False)

# --- target-strict transpilation to the selected backend ---
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)

# --- run with SamplerV2 (IBM Runtime) ---
sampler = SamplerV2(mode=backend)
res = sampler.run([isa_circuit], shots=shots).result()

# # --- compute <Z^{⊗n}> from the distribution ---
# # res[0] is typically dict-like; fall back to .data if needed
# dist = res[0] if isinstance(res[0], dict) else getattr(res[0], "data", res[0])
# exp_val = sum(((-1) ** s.count("1")) * p for s, p in dist.items())

# print(f" > Expectation value (from Sampler): {exp_val}")
# try:
#     print(f" > Metadata: {res[0].metadata}")
# except Exception:
#     pass


Simulators: []
Devices: ['ibm_brisbane', 'ibm_torino']
Using backend: ibm_brisbane


/var/folders/1g/pjkrsvtx455c4cm1_j7xrnz00000gp/T/ipykernel_26577/753434617.py:29: DeprecationWarning: The class ``qiskit.circuit.library.iqp.IQP`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. Use the qiskit.circuit.library.iqp function instead.
  circuit: QuantumCircuit = IQP(mat)


In [12]:
print(f">>> Circuit ops (ISA): {isa_circuit.count_ops()}")
print('res is ', res)

>>> Circuit ops (ISA): OrderedDict({'rz': 909, 'sx': 638, 'ecr': 312, 'x': 61, 'measure': 50, 'barrier': 1})
res is  PrimitiveResult([SamplerPubResult(data=DataBin(meas=BitArray(<shape=(), num_shots=20000, num_bits=50>)), metadata={'circuit_metadata': {}})], metadata={'execution': {'execution_spans': ExecutionSpans([DoubleSliceSpan(<start='2025-10-07 21:03:15', stop='2025-10-07 21:03:22', size=20000>)])}, 'version': 2})


In [3]:
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2
sampler = SamplerV2(mode=backend)
# Set defaults on the live sampler
sampler.options.default_shots = 2048
sampler.options.dynamical_decoupling.enable = True
sampler.options.dynamical_decoupling.sequence_type = 'XY4'
sampler.options.twirling.enable_gates = True

# Per-job override (temporarily disable twirling for this run)
job = sampler.run([qc], shots=1024)
# , options={'twirling': {'enable': False}})
print('Job submitted with per-run override; check results to confirm.')

IBMInputValueError: 'The instruction h on qubits (0,) is not supported by the target system. Circuits that do not match the target hardware definition are no longer supported after March 4, 2024. See the transpilation documentation (https://quantum.cloud.ibm.com/docs/guides/transpile) for instructions to transform circuits and the primitive examples (https://quantum.cloud.ibm.com/docs/guides/primitives-examples) to see this coupled with operator transformations.'



---



<div style="background:#001d6c;color:#fff;padding:10px 12px;border-radius:6px;margin:8px 0;font-weight:600;">
5.1.3 — Shots & measurement return
</div>

- Control shots globally (`sampler.options.default_shots`) or per call (`run(shots=...)`).
- Results expose counts/probabilities keyed by bitstrings plus metadata.
**Exam focus:** how to change shot count; how to read the result object.

**Example Code: Reading Results**

In [ ]:
job = sampler.run([qc], shots=4096)
result = job.result()
try:
    dist0 = result[0]
    print('Top items:', list(dist0.items())[:5])
except Exception as e:
    print('Result shape differs by version:', e)



---



<div style="background:#001d6c;color:#fff;padding:10px 12px;border-radius:6px;margin:8px 0;font-weight:600;">
5.1.4 — Dynamical decoupling (DD)
</div>

**Concept**  
DD inserts idle-time pulses to mitigate dephasing. Enable via `sampler.options.dynamical_decoupling.enable=True` and select a sequence (e.g., `XY4`). **Backend support required.**

**Example Code: Enable DD**

In [ ]:
sampler.options.dynamical_decoupling.enable = True
sampler.options.dynamical_decoupling.sequence_type = 'XY4'
print('DD enabled with XY4 (if backend supports).')



---



<div style="background:#001d6c;color:#fff;padding:10px 12px;border-radius:6px;margin:8px 0;font-weight:600;">
5.1.5 — Twirling
</div>

**Concept**  
Pauli twirling randomizes coherent errors, making them effectively stochastic and easier to mitigate. Often **combinable with DD**, depending on backend.

**Example Code: Enable Twirling**

In [ ]:
sampler.options.twirling.enable = True
print('Twirling enabled (subject to backend support).')



---



<div style="background:#001d6c;color:#fff;padding:10px 12px;border-radius:6px;margin:8px 0;font-weight:600;">
5.1.6 — Backend/simulator notes & quick validation
</div>

- Many options (DD, twirling) impact **hardware** runs; local ideal simulators may not reflect changes.
- To simulate realistically, mirror target device noise/coupling.
- **Validation idea:** run with/without DD/twirling and compare distributions.

**Example Code: A/B Toggle Scaffold**

In [ ]:
def run_with_options(enable_dd, enable_twirling, shots=2048):
    sampler.options.dynamical_decoupling.enable = enable_dd
    sampler.options.twirling.enable = enable_twirling
    job = sampler.run([qc], shots=shots)
    return job.result()
# r0 = run_with_options(False, False)
# r1 = run_with_options(True, False)
# r2 = run_with_options(False, True)
# r3 = run_with_options(True, True)
print('Toggle sets prepared; uncomment to compare distributions.')



---



<div style="background:#001d6c;color:#fff;padding:10px 12px;border-radius:6px;margin:8px 0;font-weight:600;">
5.1.7 — Sample Exam‑Style Questions
</div>

Provide brief context before each question, as in the client template. Answers are listed after the questions.



**Q1 (5.1.1)** In which place do default sampler options live?  
A. PUB only  
B. `sampler.options`  
C. Backend config  
D. `run()` only

**Q2 (5.1.1)** What does the sampler conceptually return?  
A. Statevector  
B. Eigenvalues  
C. Bitstring distribution  
D. Unitary

**Q3 (5.1.2)** Correct workflow?  
A. run() → set options → backend  
B. backend → sampler → set defaults → run()  
C. set defaults → sampler → run()  
D. sampler → run() → set defaults

**Q4 (5.1.2)** How to tweak behavior for one job?  
A. Edit calibrations  
B. Pass overrides in `run(...)`  
C. Reinstall  
D. Not possible

**Q5 (5.1.3)** Change shots for a single job via:  
A. `sampler.options.default_shots`  
B. `run(shots=...)`  
C. `sampler.options.time_limit`  
D. PUB parameters

**Q6 (5.1.3)** The result object typically exposes:  
A. Unitary  
B. Counts/probabilities  
C. Bloch vectors  
D. Cal data only

**Q7 (5.1.4)** DD primarily mitigates:  
A. Sampling variance  
B. Idle-time dephasing  
C. Queue wait  
D. Transpilation time

**Q8 (5.1.4)** A common DD sequence:  
A. HZH  
B. XY4  
C. T-echo  
D. SWAP3

**Q9 (5.1.5)** Twirling is for:  
A. Increase crosstalk  
B. Randomize coherent errors  
C. Remove readout errors  
D. Double shots

**Q10 (5.1.5)** DD and twirling can be:  
A. Mutually exclusive  
B. Combined (backend-dependent)  
C. Simulator-only  
D. Stabilizer-only

**Q11 (5.1.6)** True statement:  
A. DD/twirling always visible on ideal sim  
B. Often require hardware support  
C. Shots ignored on hardware  
D. Noise cannot be mirrored

**Q12 (5.1.6)** Quick validation:  
A. Compare runs with/without DD/twirling  
B. Check CPU temp  
C. Change optimizer  
D. Alter qubit frequency

**Answers:** 1‑B, 2‑C, 3‑B, 4‑B, 5‑B, 6‑B, 7‑B, 8‑B, 9‑B, 10‑B, 11‑B, 12‑A




---



**Summary**  
Default vs per‑run options, shots & results, DD & twirling concepts, and practical validation on hardware vs. simulator.